# 1. The basic assumptions (5 points) [Yevgeny]

1. Throughout the course we have assumed that data samples are independent. Explain why this assumption is important. What breaks if it does not hold and how it breaks?

2. Throughout the course we have assumed that new data points are sampled from the same distribution as the data points in the training set. Explain why this assumption is important. What breaks down if it does not hold and how it breaks?

# 2. M companies (15 points) [Yevgeny]

M companies have independently collected data and built binary classifiers for a problem that is of interest to you. You have bought both their data and the classifiers (a single classifier from each company), and now you want to select a classifier that you will use.

Let $i \in \{1, ..., M\}$ index the companies, let $S_i$ denote the data set collected by company $i$, let $n_i = |S_i|$ be the size of $S_i$, and let $\hat{h}^\ast_i$ be the classifier produced by company $i$ by using the data set $S_i$. Let $\delta \in (0, 1)$ be a confidence parameter.

1. Derive a generalization bound (a bound on $L(\hat{h}^\ast_i$)) that would hold for all the prediction rules with probability at least $1 − \delta$.

2. Explain how you would use the bound to select a prediction rule to use on new data.

Hint: you are allowed to introduce new notation, which should be clearly defined in terms of existing notation. For example, you can define $n = \sum_{i=1}^M n_i$ or $S = \cup_{i=1}^M S_i$. If you introduce convenient notation, you may be able to simplify your life (in the sense that you will need to write less). If you do not manage to come up with convenient notation, you may need to write a bit more. Your answer must be supported by a complete derivation or references to the relevant theorems in the course material. In case you use theorems from the course material, you must verify that their assumptions hold. You are not allowed to reference external material.

# 3. Learning functions on a triangle by discretization (15 points) [Yevgeny]

We would like to learn an arbitrary binary function on a triangle by discretizing the triangle into smaller triangles and assigning binary labels to the subtriangles. The partition at level $d$ is constructed by recursively splitting each triangle at level $d − 1$ into four subtriangles, as illustrated in Figure 1. We get a sample $S$ of size $n$ to learn the function. Let $\mathcal{H}_d$ be the hypothesis set of all possible label assignments to a partition at level $d$ (in Figure 1 we provide an illustration of the first three levels). Let $\mathcal{H} = \cup_{d=1}^\infty \mathcal{H}_d$ be the hypothesis set of all possible prediction rules that we consider. For a prediction rule $h$ let $d(h)$ denote the corresponding partition level, so that $h \in \mathcal{H}_{d(h)}$.

![fig1](images/figure1.png)

1. Derive a generalization bound for learning with $\mathcal{H}_d$. (I.e., a bound on $L(h)$ that holds for all $h \in \mathcal{H}_d$ with probability at least $1 − \delta$.)

2. Derive a generalization bound for learning with $\mathcal{H}$. (I.e., a bound on $L(h)$ that holds for all $h \in \mathcal{H}$ with probability at least $1 − \delta$.)

3. Explain how to use the latter bound to select a prediction rule $h \in \mathcal{H}$.

4. What is the maximal partition level $d$ for which your bound is non-vacuous?
(It is sufficient to give an order of magnitude, you do not need to make the
precise calculation.)

5. Let $\hat{h}^\ast_d$ be a prediction rule at level $d$ that achieves the minimal empirical error. Consider the bound on $L(\hat{h}^\ast_d)$ you derived earlier in Point 2 as a function of $d$. Explain which terms in the bound on $L(\hat{h}^\ast_d)$ (if any) increase with $d$ and which terms in the bound on $L(\hat{h}^\ast_d)$ (if any) decrease with $d$?

# 4. Ensemble methods and random forests (20 points) [Christian]

## 4.1 Depth of binary decision trees
Assume a training data $S$ set with n training data points. If all of these data point end up in a different leaf, what is the minimum and maximum depth of a binary decision tree build on $S$?

## 4.2 Bias-variance decomposition
In the lecture on decision trees and random forests, it is claimed on the slide "Bias-variance decomposition, noiseless case" that
$$
2\mathbb{E}_S \mathbb{E}_p\{(y − \mathbb{E}_{S'}\{h_{S'}(x)\})(\mathbb{E}_{S'}\{h_{S'}(x)\} − h_S(x))\}
$$
vanishes. Prove – using the notation of the lecture slides – that this is indeed true. The proof is very short, but you have to provide the intermediate steps and justify why each step is correct.

Note: In the proof, you need to "move expectations around". If you do so, state why this is valid. You can assume that the order of expectations can be changed in the sense that the conditions of Fubini’s theorem are fulfilled and you may change the order of double integrals. If the last sentence confuses you, ignore it and simply assume that all expectations are over discrete random variables taking only finite values.

## 4.3 Gain
The lecture on decision trees and random forests introduced the information gain
$$
G_{d,\theta}(S) = Q(S) − \frac{|L_{d,\theta}|}{|S|} Q(L_{d,\theta}) − \frac{|R_{d,\theta}|}{|S|} Q(R_{d,\theta}),
$$
where $Q$ is some impurity measure. Let’s consider a regression task and training data $S = \{(x_1, y_1), ..., (x_n, y_n)\}$ with $x_i \in \mathbb{R}^d$ and $y_i \in \mathbb{R}$ for $i = 1, ..., n$. Let the impurity measure be the mean squared error. That is, for A ⊆ S we have
$$
Q(A) = \min_{c\in\mathbb{R}} \frac{1}{|A|} \sum_{(x,y)\in A} (y − c)^2.
$$
Proof that $G_{d,\theta}(S) \ge 0$ for any choice of $d$ and $\theta$.

# 5. Convolutional Neural Networks (7 points) [Christian]
Consider two-dimensional input data with C channels. The number of channels is fixed, however, the spatial size $r \times s$ may vary. For example, think of color images of different sizes $r \times s$, which would all have $C = 3$ but different numbers of pixels.

Now you would like to implement binary logistic regression on top of these data. The classifier should produce a classification in {0, 1} for each of the r×s spatial positions, and each of these predictions should only depend on the $C$ channel values at this position (i.e., there is no spatial interaction). The classifier should be the same for each spatial position, that is, its weights/parameters should not depend on the spatial position $r \times s$.

In standard deep learning frameworks, this classification can be implemented by a single convolutional layer.

Please explain in one to three sentences how this can be done. If you show the corresponding PyTorch line, which is not required but may be helpful, you still need to explain what the code is doing.

# 6. National Forest Inventory (18 points) [Christian]

Quantification of forest biomass stocks and their dynamics is important for implementing effective climate change mitigation measures by aiding local forest management, studying processes driving af-, re-, and deforestation, and improving the accuracy of carbon accounting. Remote sensing using airborne LiDAR can be used to perform measurements of vegetation structure at large scale. The state-of-the-art for predicting forest biomass from LiDAR point clouds is to voxelize the 3D data and compute summarizing statistics of the point distribution along the vertical axis, such as mean heights, relative height quantiles, or metrics of heterogeneity. These simple statistical features, potentially combined with other features, then serve as inputs to prediction models.

The Danish National Forest Inventory (NFI) gathers information about Danish forests and computes statistics for the formation and assessment of national or regional forest programs, sustainability assessments, investment calculations for forest industry, strategic-level planning in general, and reporting to international conventions. The Danish NFI is based on a grid covering the entire land surface of the country (Nord-Larsen and Johannsen, 2016). The measurements are done at the level of subplots, where each subplots has a radius of 15 m.

We link the data from the Danish NFI with LiDAR data similar to Oehmcke et al. (2022), who propose a deep learning approach directly operating on the 3D point clouds. The main task is to predict aboveground tree biomass within a subplot from statistics of the 3D LiDAR data of the subplot. The data in this assignment were taken from 2013 to 2017 and were heavily preprocessed and cleaned for this assignment. First, features were derived from the point clouds of the single plots. Second, only plots with non-zero biomass and trees that are either all broadleaf trees or all conifer trees were selected. Finally, outliers were removed.

## 6.1 Data loading

In [1]:
import os

filename = 'NFI_filtered_cleaned.csv'
filepath = os.path.join('data', filename)

In [2]:
import os
import requests

url_data = 'https://raw.githubusercontent.com/christian-igel/ML/main/data/' + filename

if not os.access(filepath, os.R_OK):
    print('downloading', url_data)
    try:
        r = requests.get(url_data)
        r.raise_for_status()
        open(filepath, 'wb').write(r.content)
    except Exception as e:
        print('download failed:', e)

Read data

In [3]:
import pandas as pd

df = pd.read_csv(filepath)
df.to_csv(filepath, index=False)
df.describe()

,h_mean_1_,h_mean_2_,h_std_1_,h_std_2_,h_coov_1_,h_coov_2_,h_kur_1_,h_kur_2_,h_skew_1_,h_skew_2_,...,red_q50,red_q25,blue_q75,blue_q50,blue_q25,green_q75,green_q50,green_q25,BMag_ha,C_frac
count,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,...,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000
mean,7.409585,10.662752,5.105565,3.577681,1.051682,0.368356,1.838990,0.649883,0.288754,-0.315015,...,22139.944233,18636.451204,22296.699620,19473.237009,16995.629911,25513.125475,21920.365019,18655.026616,113.640865,0.468314
std,5.411430,5.781779,2.692018,1.783427,0.852839,0.140408,17.111033,4.515893,1.561340,0.909943,...,8326.425086,8049.423340,7008.356714,6988.936708,6880.897220,7293.151396,7354.904510,7232.853660,104.941748,0.499074
min,0.007347,1.117059,0.071730,0.000000,0.075114,0.000000,-1.926685,-3.000000,-13.816676,-3.747779,...,0.000000,0.000000,256.000000,0.000000,0.000000,256.000000,0.000000,0.000000,0.002059,0.000000
25%,2.906589,5.994878,3.073276,2.330300,0.567644,0.272190,-1.277054,-0.690741,-0.627047,-0.845669,...,16384.000000,12800.000000,17600.000000,14848.000000,12544.000000,20224.000000,16640.000000,13568.000000,28.934917,0.000000
50%,6.438852,10.085121,4.823045,3.371879,0.835525,0.352517,-0.623700,-0.122410,0.095032,-0.389580,...,22016.000000,18432.000000,22016.000000,19200.000000,16640.000000,25344.000000,21760.000000,18688.000000,86.230206,0.000000
75%,10.894731,14.578052,6.953171,4.658597,1.272893,0.435651,1.014367,0.883007,0.947310,0.161931,...,28160.000000,24320.000000,26368.000000,23552.000000,20992.000000,30272.000000,26880.000000,23552.000000,169.278397,1.000000
max,31.577832,31.654262,14.545132,12.496709,13.132664,1.359926,573.404330,205.440694,18.965810,7.757708,...,51200.000000,51200.000000,53376.000000,50176.000000,50176.000000,53376.000000,47104.000000,44288.000000,956.210950,1.000000


Define relevant features. Do not use other features.

In [4]:
regression_target = 'BMag_ha'
classification_target = 'C_frac'
features = ['h_mean_1_', 'h_mean_2_', 'h_std_1_', 'h_std_2_', 'h_coov_1_', 'h_coov_2_', 'h_skew_1_', 'h_skew_2_',
            'IR_', 'h_q5_1_', 'h_q10_1_', 'h_q25_1_', 'h_q50_1_', 'h_q75_1_', 'h_q90_1_', 'h_q95_1_', 'h_q99_1_', 'h_q5_2_',
            'h_q10_2_', 'h_q25_2_', 'h_q50_2_', 'h_q75_2_', 'h_q90_2_', 'h_q95_2_', 'h_q99_2_', 'red_q75', 'red_q50', 'red_q25',
            'blue_q75', 'blue_q50', 'blue_q25', 'green_q75', 'green_q50', 'green_q25']

The data comes in a training, test, and validation split. Combine training- and validation set into a single data frame, for tasks that don't require a validation set.

In [5]:
# Define set with train and validation data
df_trainval = df.query("split in ['train', 'val']")

# Define set with test data
df_test = df.query("split == 'test'")

# Select only the features we want to use
X_trainval = df_trainval[features]
X_test = df_test[features]

# Convert target values from dataframes to numpy arrays
y_trainval = df_trainval[[regression_target]].values.ravel()
y_test = df_test[[regression_target]].values.ravel()

The property `C_frac` is not used as an input feature. It give the fraction of conifer trees in the subplot. The data are filtered so that `C_frac` is either zero or one. This feature can be used to classify the subplot into broadleaf or confier.

## 6.2 Regression

The task is to evaluate regression approaches on the data. The regression target is the aboveground biomass `BMag_ha`.

Build the models using the training and validation data. The test data must only be used for final evaluation.

### 1. Apply linear Regression without regularization. Report the test $R^2$ score.

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Define linear regression model
lr_model = LinearRegression()

# Fit model on TrainVal data
lr_model.fit(X_trainval, y_trainval)

# Predict on TrainVal and Test data
lr_trainval_y_pred = lr_model.predict(X_trainval)
lr_test_y_pred = lr_model.predict(X_test)

# Print R2 scores
print(f"R2 score (training): {r2_score(y_trainval, lr_trainval_y_pred):.3f}")
print(f"R2 score (test): {r2_score(y_test, lr_test_y_pred):.3f}")

R2 score (training): 0.712
R2 score (test): 0.739


### 2. Apply random forest with 100 and 1000 trees. Use the OOB score to determine which number of trees is better. Report OOB scores of both models and the test $R^2$ score of the better solution.

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Define parameter grid
rf_param_grid = {
    'n_estimators': [100, 1000],
}

# Define random forest model
rf_model = RandomForestRegressor(random_state=0, oob_score=True)

# Grid search
rf_grid_search = GridSearchCV(rf_model, param_grid=rf_param_grid, cv=3, n_jobs=-1, verbose=3)

# Fit model with grid search on TrainVal data
rf_grid_search.fit(X_trainval, y_trainval)

# Best model from grid search
rf_best_model = rf_grid_search.best_estimator_

# Predict with best model on TrainVal and Test data
rf_best_model_trainval_y_pred = rf_best_model.predict(X_trainval)
rf_best_model_test_y_pred = rf_best_model.predict(X_test)

# Print best number of estimators
print(f"Best number of estimators: {rf_grid_search.best_params_['n_estimators']}")
# Print OOB score
print(f"OOB score: {rf_best_model.oob_score_:.3f}")
# Print R2 scores
print(f"R2 score (training): {r2_score(y_trainval, rf_best_model_trainval_y_pred):.3f}")
print(f"R2 score (test): {r2_score(y_test, rf_best_model_test_y_pred):.3f}")

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Best number of estimators: 1000
OOB score: 0.698
R2 score (training): 0.902
R2 score (test): 0.731


### Apply $k$-nearest-neighbor regression. Find out if normalization to zero mean and unit variance helps. Describe how you did that. Use cross-validation to determine the number of neighbors from the range {1, 3, 5, ..., 47, 49}. Describe how you determined the number of neighbors. Report training and test $R^2$ score of the best model.

Unnormalized

In [8]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score

# Define parameter grid
knn_param_grid = {'n_neighbors': list(range(1, 50, 2))} # odd numbers from 1 to 50 (majority vote, no ties)

# Define kNN model and grid search
knn_grid_search = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=knn_param_grid, cv=5, n_jobs=-1)

# Fit model with grid search on training set
knn_grid_search.fit(X_trainval, y_trainval)

# Best model from grid search
knn_best_unnormalized_model = knn_grid_search.best_estimator_

# Predict with best model on TrainVal and Test data
knn_best_unnormalized_model_trainval_y_pred = knn_best_unnormalized_model.predict(X_trainval)
knn_best_unnormalized_model_test_y_pred = knn_best_unnormalized_model.predict(X_test)

# Print best number of neighbors
print(f"Best number of neighbors: {knn_grid_search.best_params_['n_neighbors']}")
# Print R2 score for training set and test set
print(f"R2 score (training): {r2_score(y_trainval, knn_best_unnormalized_model_trainval_y_pred):.3f}")
print(f"R2 score (test): {r2_score(y_test, knn_best_unnormalized_model_test_y_pred):.3f}")

Best number of neighbors: 33
R2 score (training): 0.135
R2 score (test): 0.027


Normalized

In [9]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score
from sklearn.preprocessing import StandardScaler

# Assuming X_train and X_test are your training and test datasets
scaler = StandardScaler()
X_trainval_normalized = scaler.fit_transform(X_trainval)
X_test_normalized = scaler.transform(X_test)

# Define parameter grid
knn_param_grid = {'n_neighbors': list(range(1, 50, 2))}  # {1, 3, 5, ..., 49}

# Define kNN model
knn_model = KNeighborsRegressor()

# Grid search
knn_grid_search = GridSearchCV(estimator=knn_model, param_grid=knn_param_grid, cv=5, n_jobs=-1)

# Fit model with grid search on training set
knn_grid_search.fit(X_trainval_normalized, y_trainval)

# Best model from grid search
knn_best_normalized_model = knn_grid_search.best_estimator_

# Predict with best model on TrainVal and Test data
knn_best_normalized_model_trainval_y_pred = knn_best_normalized_model.predict(X_trainval_normalized)
knn_best_normalized_model_test_y_pred = knn_best_normalized_model.predict(X_test_normalized)

# Print best number of neighbors
print(f"Best number of neighbors: {knn_grid_search.best_params_['n_neighbors']}")
# Print R2 score for training set and test set
print(f"R2 score (training): {r2_score(y_trainval, knn_best_normalized_model_trainval_y_pred):.3f}")
print(f"R2 score (test): {r2_score(y_test, knn_best_normalized_model_test_y_pred):.3f}")

Best number of neighbors: 41
R2 score (training): 0.706
R2 score (test): 0.732


# 7. Clustering (20 points) [Sadegh]

Consider a dataset consisting of 5 data points $A,B,C,D,E$ as shown in Figure 2. The pair next to each point shows the $x$ and $y$ coordinates of each data point. We wish to group the datapoints into 3 clusters according to the k-means criterion, and using the `k-means++` algorithm.

![fig2](images/figure2.png)


In [ ]:
import numpy as np

data = {
    'A': np.array([25., 0.]),
    'B': np.array([0., 3.]),
    'C': np.array([0., -3.]),
    'D': np.array([-25., 1.]),
    'E': np.array([-25., -1.]),
}

## 1. How does the algorithm choose the first initial cluster center $c_1$? (In other words, determine the probability of each data point being chosen as $c_1$).

## 2. Conditioned on $C$ being chosen as $c_1$, how does the algorithm choose $c_2$? Report the associated probability distribution.

In [38]:
import numpy as np
from tabulate import tabulate

def kmeans_plusplus_initialization(data, initial_centroids=[]):
    centroids_indices = initial_centroids.copy()
    total_centroids = len(initial_centroids) + 1

    for _ in range(total_centroids - len(initial_centroids)):
        squared_distances = []
        nearest_distances = []

        # Distance D(x) for each data point
        for point_name in data:
            distances_to_centroids = [np.linalg.norm(data[point_name] - data[c]) for c in centroids_indices]
            nearest_distance = min(distances_to_centroids)
            nearest_distances.append(nearest_distance)
            squared_distances.append(nearest_distance ** 2)

        # Weighted probability
        total_squared_distance = sum(squared_distances)
        probabilities = [d / total_squared_distance for d in squared_distances]
        
        # Print distances and probabilities for the next cluster center
        print(f"\nDistances and Probabilities for choosing cluster center C{len(centroids_indices) + 1}:")
        table_data = []
        for point_name, dist, prob in zip(data.keys(), nearest_distances, probabilities):
            table_data.append([point_name, dist, prob])
        headers = ["Point", "Distance", "Probability"]
        print(tabulate(table_data, headers=headers, floatfmt=(".3f", ".3f", ".3f")))

        # Choose new center with weighted probability
        next_centroid = np.random.choice(list(data.keys()), p=probabilities)
        centroids_indices.append(next_centroid)

In [39]:
kmeans_plusplus_initialization(data, ['C'])


Distances and Probabilities for choosing cluster center C2:
Point      Distance    Probability
-------  ----------  -------------
A            25.179          0.327
B             6.000          0.019
C             0.000          0.000
D            25.318          0.330
E            25.080          0.324


## 3. Conditioned on $c_2 = A$ and $c_1 = C$, how does the algorithm choose $c_3$? Report the associated probability distribution.

In [40]:
kmeans_plusplus_initialization(data, ['C', 'A'])


Distances and Probabilities for choosing cluster center C3:
Point      Distance    Probability
-------  ----------  -------------
A             0.000          0.000
B             6.000          0.028
C             0.000          0.000
D            25.318          0.491
E            25.080          0.482


## 4. Suppose that the algorithm chooses $c_3 = D$ --- hence, $c_1 = C$, $c_2 = A$, $c_3 = D$. Determine the final clustering returned by `k-means++` based on these initial cluster centers (i.e., determine the final clustering and the associated cluster centers). Also argue how you determined that the algorithm has converged. Further, compute the corresponding cost $\phi_1 := \phi(\mathcal{X}, \mathcal{C})$.

In [15]:
import numpy as np
from tabulate import tabulate

def kmeans(data, initial_centroids):
    centroids = np.array([data[point] for point in initial_centroids])
    prev_centroids = np.zeros(centroids.shape)
    clusters = {i: [] for i in range(len(centroids))}
    iteration = 1
    
    # Continue until centroids converge
    while not np.array_equal(centroids, prev_centroids):
        print(f"\nIteration {iteration}:\n{'-' * 20}")
        clusters = {i: [] for i in range(len(centroids))}

        # Assignment step
        table_data = []
        for point_name, point_coords in data.items():
            distances = np.linalg.norm(point_coords - centroids, axis=1)
            cluster_index = np.argmin(distances)
            clusters[cluster_index].append(point_name)
            table_data.append([point_name] + list(distances))
        headers = ["Point"] + [f"c{i+1}" for i in range(len(centroids))]
        print(tabulate(table_data, headers=headers, floatfmt=".3f"))        

        # Update step
        prev_centroids = centroids.copy()
        for i, point_names in clusters.items():
            if point_names:
                new_centroid = np.mean([data[pn] for pn in point_names], axis=0)
                centroids[i] = new_centroid
                print(f"Updated cluster center, c{i+1}: from [{prev_centroids[i][0]:.3f}, "
                      f"{prev_centroids[i][1]:.3f}] to [{new_centroid[0]:.3f}, {new_centroid[1]:.3f}]")

        for i, point_names in clusters.items():
            print(f"Cluster c{i+1}: {point_names}")
        
        iteration += 1
    
    # Display final results
    print("\nFinal Results:\n" + '-' * 20)
    for i, point_names in clusters.items():
        print(f"Cluster c{i+1}: {point_names}")
    for i, centroid in enumerate(centroids):
        formatted_centroid = [f"{element:.3f}" for element in centroid]
        print(f"c{i+1} = [{', '.join(formatted_centroid)}]")

In [12]:
kmeans(data, ['C', 'A', 'D'])


Iteration 1:
--------------------
Point        c1      c2      c3
-------  ------  ------  ------
A        25.179   0.000  50.010
B         6.000  25.179  25.080
C         0.000  25.179  25.318
D        25.318  50.010   0.000
E        25.080  50.010   2.000
Updated cluster center, c1: from [0.000, -3.000] to [0.000, 0.000]
Updated cluster center, c2: from [25.000, 0.000] to [25.000, 0.000]
Updated cluster center, c3: from [-25.000, 1.000] to [-25.000, 0.000]
Cluster c1: ['B', 'C']
Cluster c2: ['A']
Cluster c3: ['D', 'E']

Iteration 2:
--------------------
Point        c1      c2      c3
-------  ------  ------  ------
A        25.000   0.000  50.000
B         3.000  25.179  25.179
C         3.000  25.179  25.179
D        25.020  50.010   1.000
E        25.020  50.010   1.000
Updated cluster center, c1: from [0.000, 0.000] to [0.000, 0.000]
Updated cluster center, c2: from [25.000, 0.000] to [25.000, 0.000]
Updated cluster center, c3: from [-25.000, 0.000] to [-25.000, 0.000]
Cluster c

## 5. Now suppose that we had $c_1 = D$, $c_2 = A$, $c_3 = E$. Repeat the previous part with these initial cluster centers.

In [13]:
kmeans(data, ['D', 'A', 'E'])


Iteration 1:
--------------------
Point        c1      c2      c3
-------  ------  ------  ------
A        50.010   0.000  50.010
B        25.080  25.179  25.318
C        25.318  25.179  25.080
D         0.000  50.010   2.000
E         2.000  50.010   0.000
Updated cluster center, c1: from [-25.000, 1.000] to [-12.500, 2.000]
Updated cluster center, c2: from [25.000, 0.000] to [25.000, 0.000]
Updated cluster center, c3: from [-25.000, -1.000] to [-12.500, -2.000]
Cluster c1: ['B', 'D']
Cluster c2: ['A']
Cluster c3: ['C', 'E']

Iteration 2:
--------------------
Point        c1      c2      c3
-------  ------  ------  ------
A        37.553   0.000  37.553
B        12.540  25.179  13.463
C        13.463  25.179  12.540
D        12.540  50.010  12.855
E        12.855  50.010  12.540
Updated cluster center, c1: from [-12.500, 2.000] to [-12.500, 2.000]
Updated cluster center, c2: from [25.000, 0.000] to [25.000, 0.000]
Updated cluster center, c3: from [-12.500, -2.000] to [-12.500, -2.000

## 6. Letting $\phi'$ denote the cost in the previous part, what is the approximation ratio $\phi'/\phi^\star$. (In this example, $\phi^\starϕ = 20$). Explain whether or not your calculated ratio agrees with the theoretical guarantee of `k-means++` (see slides).